In [1]:
import pandas as pd
import numpy as np
import math


import warnings
warnings.filterwarnings("ignore")

In [2]:
gepland= pd.read_csv('geplande_ritten.csv', sep=';') # weekly planned pickups by PostNL, these planned pickups are the same the whole year
incident = pd.read_csv('incidentele_ritten.csv', sep=';') # additional vans required and hired by PostNL
prophpred = pd.read_csv('test_forecast_nolock_prophet.csv') # predicted by Prophet
actual = pd.read_csv('newdata.csv') # actual volume values

actual= actual[['datum', 'pc3', 'volume_per_pc3' ]]

In [3]:
prophpred['ds']=pd.to_datetime(prophpred['ds'])
prophpred=(prophpred.loc[prophpred['ds'] >= '2022-10-07']).reset_index(drop=True)

In [4]:
incident['Orderdatum']=pd.to_datetime(incident['Orderdatum'])
prophpred['ds']=pd.to_datetime(prophpred['ds'])
actual['datum']=pd.to_datetime(actual['datum'])

incident = (incident.loc[incident['Orderdatum'] >= '2022-10-07'])
incident = (incident.loc[incident['Orderdatum'] < '2022-11-07']).reset_index(drop=True)

In [5]:
# converting pc6 to pc3 for planned dataset
post1= []
for i in gepland['Postcode']:
    post1.append( i[:3])
    
gepland['pc3']= post1

# converting pc6 to pc3 for incidental pl
post2= []
for i in incident['Postcode']:
    post2.append( i[:3])
    
incident['pc3']= post2

# onrelevante kolommen droppen
gepland.drop(columns=['Type contract','Postcode', 'Recordtype', 'Startdatum', 'Einddatum', 'Vervoerder', 'Number of Mailbags', 'SYSHBSTMPLVolume' ],  axis=1, inplace=True)
incident.drop(columns=[ 'Ordertype', 'Postcode', 'Datum van activering', 'RecordTypeValue', 'Status'  ],  axis=1, inplace=True)

# kolom ritten toevoegen
incident['extra_ritten']=1

In [6]:
# data samenvoegen en nieuwe kolom namen voor weekdagen
gepland=gepland.groupby(['pc3']).agg({'Ma':'sum','Di':'sum', 'Wo':'sum','Do':'sum', 'Vr':'sum','Za':'sum','Zo':'sum'}).reset_index()
gepland.rename({'Ma':'Monday','Di':'Tuesday', 'Wo':'Wednesday','Do':'Thursday', 'Vr':'Friday','Za':'Saturday','Zo':'Sunday'}, axis=1, inplace=True)

display (gepland, prophpred, incident)

,pc3,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,100,15,15,15,15,15.0,15,0
1,101,154,159,159,159,159.0,90,33
2,102,52,33,36,36,36.0,28,2
3,103,90,92,92,92,92.0,58,11
4,104,11,11,8,11,11.0,6,0
...,...,...,...,...,...,...,...,...
834,995,18,16,16,16,16.0,6,3
835,996,23,20,20,23,23.0,12,1
836,997,9,12,12,12,12.0,6,0
837,998,21,18,18,18,18.0,10,1


,ds,pc,yhat,yhat_upper,yhat_lower
0,2022-10-07,101,36.768030,45.350999,28.116848
1,2022-10-08,101,20.076871,28.733801,11.214192
2,2022-10-09,101,1.781364,10.493470,-6.947426
3,2022-10-10,101,39.706604,48.188031,30.830976
4,2022-10-11,101,38.915193,47.302944,30.415054
...,...,...,...,...,...
24361,2022-11-02,999,0.912457,1.336803,0.491836
24362,2022-11-03,999,0.855301,1.264605,0.444626
24363,2022-11-04,999,0.930991,1.365776,0.516341
24364,2022-11-05,999,0.480637,0.895977,0.088912


,Orderdatum,pc3,extra_ritten
0,2022-11-01,416,1
1,2022-11-01,103,1
2,2022-11-01,396,1
3,2022-11-01,532,1
4,2022-11-01,695,1
...,...,...,...
3160,2022-10-31,826,1
3161,2022-10-31,470,1
3162,2022-10-31,504,1
3163,2022-10-31,170,1


In [7]:
# er kunnen 4 m3 pakketen per busje mee, 1 busje per PC area
#  maak kolom in proph met hoeveel busjes nodig zijn per pc

voorspeld =[]
for i in prophpred['yhat']:
    voorspeld.append( math.ceil(i/4))
prophpred['ritten_voorspelling'] = voorspeld
prophpred.drop(columns=[ 'yhat_upper', 'yhat_lower', 'yhat' ],  axis=1, inplace=True)


nodig =[]
for i in actual['volume_per_pc3']:
    nodig.append( math.ceil(i/4))
actual['ritten_nodig'] = nodig

In [8]:
# preparing datasets to merge so renaming columns
prophpred.rename({'ds':'datum','pc':'pc3'}, axis=1, inplace=True)
incident.rename({'Orderdatum':'datum'}, axis=1, inplace=True)

In [9]:
gepland= gepland.melt(id_vars=['pc3'], 
              value_vars=['Monday', 'Tuesday','Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday' ],
              var_name='weekday', value_name='geplande_ritten')

In [10]:
gepland.sort_values(['pc3'], inplace = True)
prophpred.sort_values(['datum', 'pc3'], inplace = True)
incident.sort_values(['datum', 'pc3'], inplace = True)

display (gepland, prophpred, incident, actual)

,pc3,weekday,geplande_ritten
0,100,Monday,15.0
2517,100,Thursday,15.0
3356,100,Friday,15.0
839,100,Tuesday,15.0
1678,100,Wednesday,15.0
...,...,...,...
3355,999,Thursday,13.0
2516,999,Wednesday,13.0
1677,999,Tuesday,13.0
838,999,Monday,12.0


,datum,pc3,ritten_voorspelling
0,2022-10-07,101,10
31,2022-10-07,102,2
62,2022-10-07,103,4
93,2022-10-07,104,0
124,2022-10-07,105,9
...,...,...,...
24241,2022-11-06,995,1
24272,2022-11-06,996,1
24303,2022-11-06,997,0
24334,2022-11-06,998,0


,datum,pc3,extra_ritten
583,2022-10-07,613,1
584,2022-10-07,732,1
660,2022-10-08,103,1
649,2022-10-08,106,1
624,2022-10-08,108,1
...,...,...,...
580,2022-11-06,529,1
574,2022-11-06,530,1
578,2022-11-06,687,1
571,2022-11-06,758,1


,datum,pc3,volume_per_pc3,ritten_nodig
0,2020-01-01,101,0.155,1
1,2020-01-02,101,26.745,7
2,2020-01-03,101,32.760,9
3,2020-01-04,101,21.934,6
4,2020-01-05,101,5.924,2
...,...,...,...,...
818221,2022-11-02,999,0.744,1
818222,2022-11-03,999,1.345,1
818223,2022-11-04,999,0.747,1
818224,2022-11-05,999,0.354,1


In [11]:
incident['pc3'] = incident['pc3'].astype("int64")
gepland['pc3'] = gepland['pc3'].astype("int64")


newdf = prophpred.merge(incident,  how='left', on=['pc3','datum'])
newdf["extra_ritten"].fillna(0, inplace = True)
newdf['weekday'] = newdf['datum'].dt.day_name()

In [12]:
newdf

,datum,pc3,ritten_voorspelling,extra_ritten,weekday
0,2022-10-07,101,10,0.0,Friday
1,2022-10-07,102,2,0.0,Friday
2,2022-10-07,103,4,0.0,Friday
3,2022-10-07,104,0,0.0,Friday
4,2022-10-07,105,9,0.0,Friday
...,...,...,...,...,...
25110,2022-11-06,995,1,0.0,Sunday
25111,2022-11-06,996,1,0.0,Sunday
25112,2022-11-06,997,0,0.0,Sunday
25113,2022-11-06,998,0,0.0,Sunday


In [13]:
newdf =newdf.merge(gepland,how='left', on=['pc3','weekday']  )
newdf =newdf.merge(actual, how='left', on=['pc3','datum'] )
newdf['plaatsgevonden_ritten']= (newdf['extra_ritten'] + newdf['geplande_ritten'] )

In [14]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25115 entries, 0 to 25114
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   datum                  25115 non-null  datetime64[ns]
 1   pc3                    25115 non-null  int64         
 2   ritten_voorspelling    25115 non-null  int64         
 3   extra_ritten           25115 non-null  float64       
 4   weekday                25115 non-null  object        
 5   geplande_ritten        25022 non-null  float64       
 6   volume_per_pc3         25115 non-null  float64       
 7   ritten_nodig           25115 non-null  int64         
 8   plaatsgevonden_ritten  25022 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 1.9+ MB


In [21]:
newdf.drop(columns=[ 'weekday', 'volume_per_pc3', 'geplande_ritten', 'extra_ritten'],  axis=1, inplace=True)
newdf

,datum,pc3,ritten_voorspelling,ritten_nodig,plaatsgevonden_ritten
0,2022-10-07,101,10,9,159.0
1,2022-10-07,102,2,2,36.0
2,2022-10-07,103,4,4,92.0
3,2022-10-07,104,0,1,11.0
4,2022-10-07,105,9,8,170.0
...,...,...,...,...,...
25110,2022-11-06,995,1,1,3.0
25111,2022-11-06,996,1,1,1.0
25112,2022-11-06,997,0,0,0.0
25113,2022-11-06,998,0,1,1.0


In [25]:
# postnl geplande teveel gestuurde ritten
postnl_ritten_teveel = 0
for index, row in newdf.iterrows():
    if row['plaatsgevonden_ritten'] > row['ritten_nodig']:
        postnl_ritten_teveel += (row['plaatsgevonden_ritten']-  row['ritten_nodig'] )
        
# Prophet  teveel voorspelde ritten

model_ritten_teveel = 0
for index, row in newdf.iterrows():
    if row['ritten_voorspelling'] > row['ritten_nodig']:
        model_ritten_teveel += (row['ritten_voorspelling']-  row['ritten_nodig'] )   
        
print ('door postnl te veel geplande_ritten',postnl_ritten_teveel )
print ('door model te veel voorspelde ritten',model_ritten_teveel )
print ('ritten bespaard door model', (postnl_ritten_teveel-model_ritten_teveel) )        

door postnl te veel geplande_ritten 595387.0
door model te veel voorspelde ritten 4057
ritten bespaard door model 591330.0


In [27]:
# postnl geplande te weinig ritten
postnl_ritten_teweinig = 0
for index, row in newdf.iterrows():
    if row['plaatsgevonden_ritten'] < row['ritten_nodig']:
        postnl_ritten_teweinig += (row['ritten_nodig']-row['plaatsgevonden_ritten'] )
        
# Model te weinig voorspelde ritten
model_ritten_teweinig = 0
for index, row in newdf.iterrows():
    if row['ritten_voorspelling'] < row['ritten_nodig']:
        model_ritten_teweinig += (row['ritten_nodig']-row['ritten_voorspelling'] )
        
print ('door postnl te weinig geplande_ritten',postnl_ritten_teweinig )
print ('door model te weinig voorspelde ritten',model_ritten_teweinig )
print ('ritten bespaard door model', (model_ritten_teweinig-postnl_ritten_teweinig) )        

door postnl te weinig geplande_ritten 408.0
door model te weinig voorspelde ritten 2352
ritten bespaard door model 1944.0
